# Imports globales y funciones de utilidad

In [ ]:
import time
import torch
import itertools
import torchvision
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision import transforms
from torchvision.datasets import CIFAR10
from torch.utils.data.dataloader import DataLoader
from sklearn.metrics import accuracy_score, confusion_matrix


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
def get_dataloaders(train_transf, test_transf, batch_size):

  train_dataset = CIFAR10("data", train=True, download=True, transform=train_transf)
  test_dataset = CIFAR10("data", train=False, download=True, transform=test_transf)

  train_size = int(0.8 * len(train_dataset))
  valid_size = len(train_dataset) - train_size
  train_dataset, validation_dataset = torch.utils.data.random_split(train_dataset, [train_size, valid_size])

  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=4)
  valid_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=4)
  test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=4)

  return train_loader, valid_loader, test_loader

# Image Augmentation

La primera parte de este laboratorio consiste en expandir un poco el modelo de LeNet para tener más parámetros y luego explorar algunas técnicas de Image Augmentation.

El modelo a implementar es el siguiente:


![Image](https://i.ibb.co/WxGgbmL/Capture.png)


In [ ]:
class CustomCNN(nn.Module):
  def __init__(self, in_channels):
    # in_channels: int, cantidad de canales de la imagen original
    super(CustomCNN, self).__init__()
    # Su implementacion
    self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=32, kernel_size=3, stride=1, padding=1)
    self.conv2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1)
    self.pooling = nn.MaxPool2d(kernel_size=2, stride=2)
    self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
    self.conv4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)
    self.conv5 = nn.Conv2d(in_channels=64, out_channels=120, kernel_size=3, stride=1, padding=1)
    self.fc1 = nn.Linear(in_features=4*4*120, out_features=512)
    self.output = nn.Linear(in_features=512, out_features=10)
    self.dropout = nn.Dropout(p=0.5)


  def forward(self, x):
    result = F.relu(self.conv1(x))
    result = F.relu(self.conv2(result))
    result = self.pooling(result)
    
    result = F.relu(self.conv3(result))
    result = F.relu(self.conv4(result))
    result = self.pooling(result)

    result = F.relu(self.conv5(result))
    result = self.pooling(result)
    
    result = result.flatten(1)
    result = self.dropout(result)

    result = F.relu(self.fc1(result))
    result = self.dropout(result)

    result = self.output(result)

    return result

## Funciones genericas para entrenar nuestros modelos

Vamos a utilizar las mismas funciones que implementamos en los laboratorios anteriores para entrenar y testear nuestros modelos.

In [ ]:
def train_epoch(training_model, loader, criterion, optim):
    training_model.train()
    epoch_loss = 0.0
    all_labels = []
    all_predictions = []
    
    for images, labels in loader:
      all_labels.extend(labels.numpy())  

      optim.zero_grad()

      predictions = training_model(images.to(device))
      all_predictions.extend(torch.argmax(predictions, dim=1).cpu().numpy())

      loss = criterion(predictions, labels.to(device))
      
      loss.backward()
      optim.step()

      epoch_loss += loss.item()

    return epoch_loss / len(loader), accuracy_score(all_labels, all_predictions) * 100


def validation_epoch(val_model, loader, criterion):
    val_model.eval()
    epoch_loss = 0.0
    all_labels = []
    all_predictions = []
    
    with torch.no_grad():
      for images, labels in loader:
        all_labels.extend(labels.numpy())  

        predictions = val_model(images.to(device))
        all_predictions.extend(torch.argmax(predictions, dim=1).cpu().numpy())

        loss = criterion(predictions, labels.to(device))

        epoch_loss += loss.item()

    return epoch_loss / len(loader), accuracy_score(all_labels, all_predictions) * 100
  

def train_model(model, train_loader, test_loader, criterion, optim, number_epochs):
  train_history = []
  test_history = []
  accuracy_history = []

  for epoch in range(number_epochs):
      start_time = time.time()

      train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer)
      train_history.append(train_loss)
      print("Training epoch {} | Loss {:.6f} | Accuracy {:.2f}% | Time {:.2f} seconds"
            .format(epoch + 1, train_loss, train_acc, time.time() - start_time))

      start_time = time.time()
      test_loss, acc = validation_epoch(model, test_loader, criterion)
      test_history.append(test_loss)
      accuracy_history.append(acc)
      print("Validation epoch {} | Loss {:.6f} | Accuracy {:.2f}% | Time {:.2f} seconds"
            .format(epoch + 1, test_loss, acc, time.time() - start_time))

## Entrenando modelos 

Comenzamos definiendo una seccion de código con valores por defecto de hiperparámetros que vamos a utilizar y luego entrenamos un modelo de la CNN definida anteriormente sin usar augmentation en los datos y otro haciendo uso del mismo.


In [ ]:
# Global models config

BATCH_SIZE = 32
LR = 0.001
NUMBER_EPOCHS = 10
criterion = nn.CrossEntropyLoss().to(device)

In [ ]:
# Fijamos las semillas siempre para poder comparar.

torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Creamos los dataloaders
test_transform = transforms.Compose([
    transforms.ToTensor()
])

train_transform = transforms.Compose([
    transforms.ToTensor()
])

train_loader, valid_loader, test_loader = get_dataloaders(train_transform, test_transform, BATCH_SIZE)

# Definimos el modelo y el optimizador
modelo_sin_aug = CustomCNN(3).to(device)
optimizer = torch.optim.Adam(modelo_sin_aug.parameters(), lr=LR)

# Entrenamos
train_model(modelo_sin_aug, train_loader, valid_loader, criterion, optimizer, NUMBER_EPOCHS)

In [ ]:
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Creamos los datasets
test_transform = transforms.Compose([
    transforms.ToTensor()
])

# Definir transormaciones que vamos a aplicar al set de entrenamiento
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])
train_loader, valid_loader, test_loader = get_dataloaders(train_transform, test_transform, BATCH_SIZE)


# Crear el modelo, optimizador y entrenarlo
modelo_con_aug = CustomCNN(3).to(device)
optimizer = torch.optim.Adam(modelo_con_aug.parameters(), lr=LR)

train_model(modelo_con_aug, train_loader, valid_loader, criterion, optimizer, NUMBER_EPOCHS)

## Evaluando los modelos en los datos de test

Vamos a comenzar evaluando en los datos de test normales y luego vamos a aplicar distintas transformaciones (para simular entornos más reales de datos) y vamos a ver la performance y robustez de los modelos que entrenamos anteriormente.

Primero agregar horizontal flip obligatorio(p=1) y luego vertical flip obligatorio, qué pasa con los modelos?

In [ ]:
test_transform = transforms.Compose([
  transforms.ToTensor()                              
])

_, _, test_loader = get_dataloaders(None, test_transform, BATCH_SIZE)

test_loss, accuracy = validation_epoch(modelo_sin_aug, test_loader, criterion)
print(f"Modelo entrenado sin augmentation: Test set: {test_loss:.6f} Loss. Accuracy {accuracy:.2f}%")

test_loss, accuracy = validation_epoch(modelo_con_aug, test_loader, criterion)
print(f"Modelo entrenado con augmentation: Test set: {test_loss:.6f} Loss. Accuracy {accuracy:.2f}%")

Modelo entrenado sin augmentation: Test set: 0.747601 Loss. Accuracy 74.12%
Modelo entrenado con augmentation: Test set: 0.690253 Loss. Accuracy 78.57%


In [ ]:
test_transform = transforms.Compose([
  transforms.RandomHorizontalFlip(1),
  transforms.ToTensor()                              
])

_, _, test_loader = get_dataloaders(None, test_transform, BATCH_SIZE)

test_loss, accuracy = validation_epoch(modelo_sin_aug, test_loader, criterion)
print(f"Modelo entrenado sin augmentation: Test set: {test_loss:.6f} Loss. Accuracy {accuracy:.2f}%")

test_loss, accuracy = validation_epoch(modelo_con_aug, test_loader, criterion)
print(f"Modelo entrenado con augmentation: Test set: {test_loss:.6f} Loss. Accuracy {accuracy:.2f}%")


Files already downloaded and verified
Files already downloaded and verified
Modelo entrenado sin augmentation: Test set: 0.757754 Loss. Accuracy 73.87%
Modelo entrenado con augmentation: Test set: 0.750489 Loss. Accuracy 74.64%


In [ ]:
test_transform = transforms.Compose([
  transforms.RandomVerticalFlip(1),
  transforms.ToTensor()                              
])

_, _, test_loader = get_dataloaders(None, test_transform, BATCH_SIZE)

test_loss, accuracy = validation_epoch(modelo_sin_aug, test_loader, criterion)
print(f"Modelo entrenado sin augmentation: Test set: {test_loss:.6f} Loss. Accuracy {accuracy:.2f}%")

test_loss, accuracy = validation_epoch(modelo_con_aug, test_loader, criterion)
print(f"Modelo entrenado con augmentation: Test set: {test_loss:.6f} Loss. Accuracy {accuracy:.2f}%")

Files already downloaded and verified
Files already downloaded and verified
Modelo entrenado sin augmentation: Test set: 2.127755 Loss. Accuracy 31.93%
Modelo entrenado con augmentation: Test set: 1.958503 Loss. Accuracy 33.56%


# DenseNet


![Image](https://miro.medium.com/max/5164/1*_Y7-f9GpV7F93siM1js0cg.jpeg)

Link al paper original: [DenseNets](https://arxiv.org/pdf/1608.06993.pdf)

Algunas consideraciones del paper a tener en cuenta:

1. Batch normalization en los inputs de los bloques densos y las capas de transición.
2. ReLU en todos lados como funcion de activación.
3. El MLP al final de la red cuenta con una capa oculta de 512 neuronas
4. Las activaciones luego del tercer bloque denso tienen tamaño 4*4 (ejercicio, calcular a mano!)


Implementamos DenseNet para resolver el problema de CIFAR10


In [ ]:
class DenseBlock(nn.Module):
  def __init__(self, in_channels):
    super(DenseBlock, self).__init__()
    self.bn = nn.BatchNorm2d(num_features=in_channels)
    self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=32, kernel_size=3, stride=1, padding=1)
    self.conv2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1)
    self.conv3 = nn.Conv2d(in_channels=64, out_channels=32, kernel_size=3, stride=1, padding=1)
    self.conv4 = nn.Conv2d(in_channels=96, out_channels=32, kernel_size=3, stride=1, padding=1)
    self.conv5 = nn.Conv2d(in_channels=128, out_channels=32, kernel_size=3, stride=1, padding=1)
    
  def forward(self, x):
    bn = self.bn(x)

    conv1 = F.relu(self.conv1(x))

    conv2 = F.relu(self.conv2(conv1))
    c2_dense = torch.cat([conv1, conv2], 1)

    conv3 = F.relu(self.conv3(c2_dense))
    c3_dense = torch.cat([conv1, conv2, conv3], 1)

    conv4 = F.relu(self.conv4(c3_dense))
    c4_dense = torch.cat([conv1, conv2, conv3, conv4], 1)

    conv5 = F.relu(self.conv5(c4_dense))
    c5_dense = torch.cat([conv1, conv2, conv3, conv4, conv5], 1)

    return c5_dense

In [ ]:
class TransitionLayer(nn.Module):
  def __init__(self, in_channels, out_channels):
    super(TransitionLayer, self).__init__()

    self.bn = nn.BatchNorm2d(num_features=in_channels)
    self.conv = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=1)
    self.avgpool = nn.AvgPool2d(kernel_size=2, stride=2)

  def forward(self, x):
    bn = self.bn(x)
    out = F.relu(self.conv(bn))
    out = self.avgpool(out)

    return out

In [ ]:
class DenseNet(nn.Module):
	def __init__(self, n_classes):
		super(DenseNet, self).__init__()

		self.input_conv = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=7, padding=3, bias=False)

		# Make Dense Blocks
		self.denseblock1 = DenseBlock(in_channels=64)
		self.denseblock2 = DenseBlock(in_channels=128)
		self.denseblock3 = DenseBlock(in_channels=128)

		# Make transition Layers
		self.transitionLayer1 = TransitionLayer(in_channels=160, out_channels=128)
		self.transitionLayer2 = TransitionLayer(in_channels=160, out_channels=128)
		self.transitionLayer3 = TransitionLayer(in_channels=160, out_channels=64)

		# Classifier
		self.fully_connected_1 = nn.Linear(64*4*4, 512)
		self.output = nn.Linear(512, n_classes)

	def forward(self, x):
		out = F.relu(self.input_conv(x))

		out = self.denseblock1(out)
		out = self.transitionLayer1(out)

		out = self.denseblock2(out)
		out = self.transitionLayer2(out)

		out = self.denseblock3(out)
		out = self.transitionLayer3(out)
    
    out = out.flatten(1)

		out = F.relu(self.fully_connected_1(out))
		out = self.output(out)

		return out

In [ ]:
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Creamos los datasets
test_transform = transforms.Compose([
    transforms.ToTensor()
])

train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])

densenet = DenseNet(10).to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(densenet.parameters(), lr=LR)

train_loader, valid_loader, test_loader = get_dataloaders(train_transform, test_transform, BATCH_SIZE)

train_model(densenet, train_loader, valid_loader, criterion, optimizer, NUMBER_EPOCHS)

Files already downloaded and verified
Files already downloaded and verified
Training epoch 1 | Loss 1.512427 | Accuracy 45.06% | Time 32.96 seconds
Validation epoch 1 | Loss 1.223909 | Accuracy 56.30% | Time 3.10 seconds
Training epoch 2 | Loss 1.050360 | Accuracy 62.66% | Time 33.86 seconds
Validation epoch 2 | Loss 1.053350 | Accuracy 63.09% | Time 3.21 seconds
Training epoch 3 | Loss 0.837127 | Accuracy 70.53% | Time 34.00 seconds
Validation epoch 3 | Loss 0.922786 | Accuracy 68.48% | Time 3.05 seconds
Training epoch 4 | Loss 0.711929 | Accuracy 75.06% | Time 33.89 seconds
Validation epoch 4 | Loss 0.847208 | Accuracy 70.80% | Time 3.19 seconds
Training epoch 5 | Loss 0.616721 | Accuracy 78.35% | Time 33.99 seconds
Validation epoch 5 | Loss 0.810936 | Accuracy 73.61% | Time 3.14 seconds
Training epoch 6 | Loss 0.541988 | Accuracy 81.28% | Time 33.87 seconds
Validation epoch 6 | Loss 0.683050 | Accuracy 77.28% | Time 3.10 seconds
Training epoch 7 | Loss 0.473242 | Accuracy 83.52% | T

In [ ]:
test_transform = transforms.Compose([
  transforms.ToTensor()                              
])

_, _, test_loader = get_dataloaders(None, test_transform, BATCH_SIZE)

test_loss, accuracy = validation_epoch(densenet, test_loader, criterion)
print(f"DenseNet Test set: {test_loss:.6f} Loss. Accuracy {accuracy:.2f}%")

Files already downloaded and verified
Files already downloaded and verified
DenseNet Test set: 0.688945 Loss. Accuracy 78.57%
